# Loading and tidying "real" data

## Tidy data as a concept

<img src="files/tidy_rdatasci.png" width=800>

Import to note "tidy" data is not the only way to work

Pandas brief descriptor <https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf>

### Loading data

Loading data into python is actually pretty easy with pandas! First let's load the pandas module

In [ ]:
import pandas as pd

Now - let's try to load some of the data we have in the `data` directory.  First - just take a look at the data (`tb_2014.csv`) by navigating to it in your file folder and opening it in jupyter like so (I have rainbow_CSV on leading to the colors):
<img src="files/tb_2014_rainbow.png">

You could also open this in excel if you prefer - by downloading the csv and opening it.

We can see that this data is a typical spreadsheet with columns and rows. CSV stands for **C**omma **S**eparated **V**ariables.

This data is information about tuberculosis prevalance from the UN WHO from 1980-2014.  There is a `country` column, then the `iso2` and `iso3` columns which appear to be appreviations (2 and 3 letter) of the country name.  A `year` column which is self explanatory.   
Then there are columns for data - where the column name means:
- The first three letters say if it's a "new" or "old" case of TB
- The next two letters tell us what type of TB
    - rel are relapsed cases
    - ep are extrapulmonary TB
    - sn are pulmonary TB with a negative pulmonary smear
    - sp are pulmonary TB with a positive pulmonary smear
- The next letter tells us the sex of TB patients give by males (m) or females (f)
- The remaining numbers tell us the age group
    - 014 is 0-14 years old
    - 1524 is 15-24 years old
    - 2534 is 25-34 years old
    - 3544 is 35-44 years old
    - 4554 is 45-54 years old
    - 5564 is 55-64 years old
    - 65 is 65 or older

Now let's load it in to python! Pandas has a bunch of different reading and writing functions: <https://pandas.pydata.org/pandas-docs/stable/reference/io.html>  I'm going to use `read_csv` here.  I type tb_raw again afterwards so that we can see the output - this is not necessary in your own code (unless you want to).

In [ ]:
tb_raw=pd.read_csv('../data/tb_2014.csv')
tb_raw

So - we now have loaded the file into a pandas "dataframe" called tb_raw.  This is the variable name that lets us organize and access the data.   
This is certainly a useful way to look at the data, but it's in "wide" form instead of "long" or tidy from.
Each row corresponds to a country year, but the categorical observations of new/old, type of TB, sex and age in one column, and all for the same country/year in the same row.

Let's manipulate this data to "tidy" it.   
I don't think we need the extra `iso2` and `iso3` columns, so let's get rid of them, using the pandas `drop` function <https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html>


In [ ]:
tb=tb_raw.drop(columns=['iso2', 'iso3'])
tb

Next - let's `melt` the data to change it from "wide" form to "long" form. First we have to `melt` the data <https://pandas.pydata.org/docs/reference/api/pandas.melt.html> which changes the data to make it long form. We'll tell melt which of the columns are the observations, and it will assume (without us specifying) that the rest of the columns are variables.

In [ ]:
tb=tb.melt(id_vars=['country', 'year'])

In [ ]:
tb

Now even just an examination of this table above tells us that the data wasn't entered in really well. You can see that though the top "variables" are "new_sp_m014" the bottom are "newrel_f65" - the underscore separates new/old from type of TB in the top part, but not in the bottom ones.  So let's first replace all the "newrel" with "new_rel".

To do this, we have to first pull out just the column we are interested in (in this case `variable`) like so:

In [ ]:
tb['variable']

Then we need to do a "find/replace" to change all the places we see "newrel" to "new_rel".  We can do this with ".str.replace" from pandas <https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.replace.html>

In [ ]:
tb['variable']=tb['variable'].str.replace('newrel', 'new_rel')
tb

Next we have to split up that variable column into the four different parts - new/old cases, type of TB, sex, and age range.  We can first split the text by the underscores to break it into three pieces using "string split" <https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html>.  We just tell the function what we want to split by, in this case "_"


In [ ]:
tb['variable'].str.split('_', expand=True)

New Old is whether the ccase is new or old

In [ ]:
tb[['new_old', 'type', 'sex_age']]=tb['variable'].str.split('_', expand=True)
tb

Ok - so next we have to try to split up the "sex_age" column to the different values.  Since the sex is always a single character in this case "m" or "f" - so if we just slice off the first character, we should be able to get what we need. The pandas `str.slice` function can help us with this <https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.slice.html>

In [ ]:
tb['sex_age'].str.slice(start=-2)

In [ ]:
tb['sex']=tb['sex_age'].str.slice(0,1)
tb['age']=tb['sex_age'].str.slice(1)
tb

Ok - we are getting close.  We can drop the "unsplit" columns and we should probably process the age range to make it a bit more intuitive to read:

In [ ]:
tb['age'].unique()

In [ ]:
tb=tb.drop(columns=['variable', 'sex_age'])
tb['age']=tb['age'].str.replace('014', '0-14')
tb['age']=tb['age'].str.replace('1524', '15-24')
tb['age']=tb['age'].str.replace('2534', '25-34')
tb['age']=tb['age'].str.replace('3544', '35-44')
tb['age']=tb['age'].str.replace('4554', '45-54')
tb['age']=tb['age'].str.replace('5564', '55-64')
tb['age']=tb['age'].str.replace('65', '65+')
tb

Let's also say that we want to write out our "fixed" and tidy dataframe to a spreadsheet to open in excel or other.  We can do that with `to_csv` <https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html>

In [ ]:
tb.to_csv('cleaned_who.csv')

Ok - _finally_ we've got the data clean.  As is frequently the case, "cleaning" the data takes a large portion of the time, especially if the data comes from someone else: 

<img src="files/datasci_tweet.PNG" width=400>

Now let's look at the data and subset it to calculate certain statistics on it!

Let's set ourselves a problem - let's say we want to find all the countries which had a higher than median number of cases in 2000.

First, we'll make a new subset of the data where the year matches 2000 like so:

In [ ]:
x=tb['year']==2000
x

In [ ]:
tb[x]

In [ ]:
twothousand=tb[tb['year']==2000]
twothousand

Can't help but notice there are a lot of rows where the value is "NaN" - this means "Not a Number" - or - there is no data for this observation.  Let's go ahead and cull out those rows:

In [ ]:
twothousand=twothousand.dropna()
twothousand

Let's filter this further, and only keep the cases with people >35:

In [ ]:
oldtwothousand=twothousand[twothousand['age'].isin(['35-44', '45-54','55-64','65+'])]
oldtwothousand

Now let's add up the number of cases **per country**.  To do this, we need to basically group the dataframe on the `country`, then apply sum all the cases for that country in the split frame, then  recombine the resulting data.  It follows this sort of idea (from <https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html>) 

<img src="files/03.08-split-apply-combine.png" width=400>

We can implement it with `groupby` <https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html>, followed by `sum`.  With `groupby` I'm going to extract only the column I want (`value`) which will make it a "Series" instead of a full dataframe, with the countries now being the rownames or Indexes.

In [ ]:
grouped=oldtwothousand.groupby(by=['country']) ['value']
percountry_old2000=grouped.sum()
percountry_old2000

In [ ]:
oldtwothousand[oldtwothousand['country']=='Algeria']

And the median of these values is:

In [ ]:
percountry_old2000.median()

Now let's find the countries that had more than that - again using the square brackets and a conditional to select subsets of the data:

In [ ]:
morecases=percountry_old2000[percountry_old2000>percountry_old2000.median()]
morecases.index

### Exercise:
Please calculate the average case number for Brazil, Chile, and Peru for each age bracket that you have data for.